<a href="https://colab.research.google.com/github/metamatematico/Experimentos-IA/blob/master/Copia_de_Gans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

In [ ]:
print(torch.__version__)


In [ ]:


device = "cuda" if torch.cuda.is_available () else "cpu"
device

In [ ]:
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)

def output_label(label):
    output_mapping = {
                 0: "T-shirt/Top",
                 1: "Trouser",
                 2: "Pullover",
                 3: "Dress",
                 4: "Coat", 
                 5: "Sandal", 
                 6: "Shirt",
                 7: "Sneaker",
                 8: "Bag",
                 9: "Ankle Boot"
                 }
    input = (label.item() if type(label) == torch.Tensor else label)
    return output_mapping[input]

In [ ]:


class Dataset(torch.utils.data.Dataset):
  def __init__(self, trainset):
    self.imgs = torch.tensor([np.array(i[0]).flatten()/ 255.0 for i in trainset], dtype=torch.float)
    self.labels = torch.tensor([i[1]for i in trainset], dtype=torch.long, device=device)

  def __len__(self):
    return len(self.imgs)

  def __getitem__(self, ix):
    return self.imgs[ix], self.labels[ix]

train = Dataset(trainset)
len(train)        

In [ ]:
img, label = train[0]
img.shape, img.dtype, img.max(), img.min()

In [ ]:
dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)

imgs, labels = next(iter(dataloader))
imgs.shape, labels.shape

In [ ]:

def block(n_in, n_out):
  return nn.Sequential(
      nn.Linear(n_in, n_out),
      nn.ReLU(inplace=True)
  )
#Esta es la arquitectura de red neuronal que se ocupará para el generador y el discriminador. En este caso es muy sencilla pues apenas son unas capas de perceptrones en  Ganbuena.ipynb ocupo redes convolucionales en ves de capas de perceptrones
#jugando con el tamaño de entrada definiremos al discriminador y al generador
class MLP(nn.Module):
  def __init__(self,input_size, outpout_size):
    super().__init__()
    self.input_size = input_size
    self.fc1 = block(input_size, 150 )
    self.fc2 = block(150, 100)
    self.fc3 = nn.Linear(100, outpout_size)

  def forward(self, x):  #forward es un modelo secuencias sencillo
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    return x

In [ ]:
#Ahora voy a instanciar el generador con la clase MLP

n_in, n_out = 30, 28*28
generator = MLP(n_in, n_out)

In [ ]:
output = generator(torch.randn(64, 30))
output.shape

In [ ]:
plt.imshow(output[0].reshape(28,28).detach().numpy())
plt.show()